In [1]:
import os
import sys
import random
import json
import pandas as pd
import numpy as np
import geopandas as gpd

from h3 import h3
from keplergl import KeplerGl

sys.path.append('../src/')
from utils import save_pickle, load_pickle, make_h3_index

RES = 10
HEX_COLUMN = f'geo_h3_{RES}'

In [2]:
df_home_job = pd.read_csv("../data/raw/01_CLocation_July.csv")
df_shape = gpd.read_file("../data/raw/fishnet2021/fishnet2021.shp")
df_home_job.rename(columns={'zid': 'cell_zid'}, inplace=True)

df_home_job = pd.merge(df_home_job, df_shape, on = ['cell_zid'])
gdf_home_job = json.loads(gpd.GeoDataFrame(df_home_job).to_json())
df_home_job.head()

,cell_zid,customers_cnt_home,customers_cnt_job,customers_cnt_day,customers_cnt_move,geometry
0,32909,4,0,0,0,"POLYGON ((37.03351 55.14029, 37.03346 55.14478..."
1,32910,25,8,26,0,"POLYGON ((37.04135 55.14032, 37.04130 55.14481..."
2,33261,2,0,2,0,"POLYGON ((37.02562 55.14475, 37.02556 55.14924..."
3,33262,13,2,11,0,"POLYGON ((37.03346 55.14478, 37.03341 55.14927..."
4,33263,10,0,3,0,"POLYGON ((37.04130 55.14481, 37.04125 55.14930..."


In [3]:
def generate_points_in_bounds(poly, size):
    minx, miny, maxx, maxy = poly.bounds
    return [(random.uniform(minx, maxx), random.uniform(miny, maxy)) for i in range(size)]

In [ ]:
list_points = []
for idx, row in df_home_job.iterrows():
    points = generate_points_in_bounds(row['geometry'], row['customers_cnt_home'])
    list_points.append(points)
list_points = [x for sublist in list_points for x in sublist]    

In [ ]:
df_points = pd.DataFrame(list_points, columns = ['lon', 'lat'])
df_points[HEX_COLUMN] = make_h3_index(df_points, 'lat', 'lon', RES)

In [ ]:
df_customers_cnt_home_hex = df_points.groupby(HEX_COLUMN, as_index=False)['lat'].count()
df_customers_cnt_home_hex.columns = [HEX_COLUMN, 'customers_cnt_home_hex']

In [ ]:
hexagon_cover_list = []
for idx in range(0, len(gdf_home_job['features'])):
    coords = gdf_home_job['features'][idx]['geometry']['coordinates']
    coords = [np.column_stack((np.array(coords[0])[:, 1],
                                                      np.array(coords[0])[:, 0])).tolist()]
    try:
        geoJson = {'type':'Polygon','coordinates': coords}
        hexagons = list(h3.polyfill(geoJson, 10))
    except:
        geoJson = {'type':'Polygon','coordinates': coords[0]}
        hexagons = list(h3.polyfill(geoJson, 10))        
    hexagon_cover_list.append(hexagons)
hexagon_cover_list = [x for sublist in hexagon_cover_list for x in sublist]

In [ ]:
df_hexagons = pd.DataFrame(hexagon_cover_list, columns = [HEX_COLUMN])
df_hexagons = pd.merge(df_hexagons, df_customers_cnt_home_hex, how = 'left', on = HEX_COLUMN)
df_hexagons['customers_cnt_home_hex'].fillna(0, inplace=True)

In [ ]:
df_hexagons.to_csv("../data/precessed/h3_10_customers_cnt_home_hex.csv", sep = ';', index = None)

In [ ]:
map_config = load_pickle("../configs/map_config_h3.pkl")
map_1 = KeplerGl(height = 700, config = map_config)
map_1.add_data(df_hexagons, name='shape')
map_1

In [ ]:
save_pickle(map_1.config, "../configs/map_config_h3.pkl")